In [ ]:
!pip install -r requirements.txt

We'll be making use a couple of plugins to help us with our analysis. Let's go ahead and install it now:

In [ ]:
!fiftyone plugins download https://github.com/harpreetsahota204/synthetic_gui_samples_plugins

In [ ]:
!fiftyone plugins download https://github.com/ehofesmann/edit_label_attributes

## Downloading the Dataset

In this section, we'll download a dataset from Google Drive using `gdown`. The file is hosted at this URL:
https://drive.google.com/file/d/1YEhdlXuTV_nalVvKuXSHZa_JgkuIXjcG/view?usp=sharing

After downloading, we'll extract its contents to work with the data.


In [ ]:
import gdown
import zipfile

# Download the file
url = "https://drive.google.com/uc?id=1YEhdlXuTV_nalVvKuXSHZa_JgkuIXjcG"
gdown.download(url, "data.zip", quiet=False)

# Extract the contents
with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

## Loading Annotations

Now we'll load and explore the COCO4GUI annotations from the JSON file.

In [ ]:
import json
import random
from pprint import pprint

# Load the annotations
annotations_path = "./data/annotations_coco.json"
with open(annotations_path, 'r') as f:
    annotations = json.load(f)

# Print basic info about the dataset
print(f"Keys in the annotations file: {list(annotations.keys())}")
print(f"Number of images: {len(annotations['images'])}")
print(f"Number of annotations: {len(annotations['annotations'])}")
print(f"Categories: {[cat['name'] for cat in annotations['categories']]}")

In [ ]:
from PIL import Image

# Select a random image and its annotations
random_image = random.choice(annotations['images'])

image_id = random_image['id']

# Get annotations for this image
image_annotations = [a for a in annotations['annotations'] if a['image_id'] == image_id]

# Print image info and its annotations
print(f"Selected image: {random_image['file_name']}")

Image.open(f"./data/{random_image['file_name']}")


In [ ]:
print(f"Image ID: {image_id}")
print(f"Image size: {random_image['width']} x {random_image['height']}")
print(f"Number of annotations for this image: {len(image_annotations)}")

# Print one annotation as an example
if image_annotations:
    print("\nExample annotation:")
    pprint(image_annotations[0])

## Loading Data into FiftyOne

[FiftyOne](https://voxel51.com/fiftyone/) is a powerful tool for visualizing, exploring and analyzing image datasets. We'll use a specialized COCO4GUI format to load our GUI interaction dataset into FiftyOne.

The [COCO4GUI](https://github.com/harpreetsahota204/coco4gui_fiftyone) format extends the standard COCO detection format to handle GUI-specific features:

- **Dual annotation support**: Both bounding boxes for UI elements and keypoints for interaction points

- **Sequence information**: Tracking user workflows and interaction chains

- **GUI metadata**: Application, platform, and timing information

- **Rich attributes**: Task descriptions, element information, and custom metadata


In [ ]:
!curl -s https://raw.githubusercontent.com/harpreetsahota204/coco4gui_fiftyone/main/coco4gui.py -o coco4gui.py

In [ ]:
import fiftyone as fo
from coco4gui import COCO4GUIDataset

# Define paths to our dataset
dataset_dir = "/Users/harpreetsahota/workspace/visual_agents_workshop/session_2"
data_path = "data"  # Image directory
labels_path = "data/annotations_coco.json"  # COCO annotations file

# Create the dataset
gui_dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=COCO4GUIDataset,
    name="session_2_dataset",
    data_path=data_path,
    labels_path=labels_path,
    overwrite=True,
    persistent=True,
    include_sequence_info=False,  # Extract sequence info
    include_gui_metadata=True,   # Include GUI metadata
    extra_attrs=True,            # Include all attributes
)

In [ ]:
gui_dataset

Let's see what a sample looks like.

In [ ]:
sample = gui_dataset.skip(51).first()
sample

In [ ]:
# 1. Accessing all detections for a sample
print("All detections for this sample:")
print(sample.detections.detections)


In [ ]:
# 2. Accessing a specific detection by index
print("\nFirst detection:")
print(sample.detections.detections[0])

In [ ]:
# 3. Filtering detections by label
hover_detections = [d for d in sample.detections.detections if d.label == "hover"]

print(f"\nFound {len(hover_detections)} hover detections")

In [ ]:
# 4. Accessing specific attributes of a detection
if hover_detections:
    detection = hover_detections[0]
    print("\nDetection details:")
    print(f"- Label: {detection.label}")
    print(f"- Bounding box: {detection.bounding_box}")
    print(f"- Task description: {detection.task_description}")
    
    # Accessing nested attributes
    if hasattr(detection, "custom_metadata") and detection.custom_metadata:
        print(f"- Custom metadata value: {detection.custom_metadata['value']}")

# Note: FiftyOne allows you to access labels as Python objects
# with dot notation, making it easy to navigate complex label structures


## Exploring the Dataset

With the dataset loaded in FiftyOne, you can:

1. **Visualize annotations**: See both bounding boxes and keypoints in the same view

2. **Filter interactions**: Use the query interface to find specific interaction types like clicks or drags

3. **Explore sequences**: View related interactions in a workflow sequence

4. **View metadata**: See application info, platform details, and custom attributes

5. **Create views**: Focus on subsets of data for detailed analysis

Try these example queries in the FiftyOne App:

- `F("ground_truth.detections.label") == "click"` - Show only click interactions

- `F("application") == "Chrome"` - Filter by application

In [ ]:
# Example: Creating filtered views of the dataset
from fiftyone import ViewField as F

# Create a view with only click interactions
clicks = gui_dataset.filter_labels("detections", F("label") == "click")

gui_dataset.save_view("click_boxes", clicks)

print(f"Click interactions (bounding boxes): {len(clicks)}")

## Identifying and Organizing Incomplete Annotations

We can easily create views that identify and organize samples with missing annotation data.  These views help us:

1. Quickly find UI elements with incomplete annotations that need attention
   
2. Focus annotation resources on specific types of missing data
   
3. Monitor overall annotation quality as the dataset evolves
   

Saving these views creates shortcuts to these filtered datasets that can be accessed through the UI or programmatically, making dataset curation much more efficient.


-----

Notice we are using `filter_labels` to filter the labels within each sample. 

This filters the contents of label fields within each sample. 

It removes or keeps only the labels that match a given filter, but all samples remain in the view (unless you set `only_matches=True`, which will exclude samples with no matching labels). 

In [ ]:
from fiftyone import ViewField as F

# Create a view with missing task descriptions
no_bb_tasks = gui_dataset.filter_labels("detections", F("task_description") == "")

gui_dataset.save_view("no_bb_tasks", no_bb_tasks)

In [ ]:
from fiftyone import ViewField as F

# Create a view with missing task descriptions
no_bb_tasks_w_match = gui_dataset.filter_labels("detections", F("task_description") == "", only_matches=True)

gui_dataset.save_view("no_bb_tasks_w_match", no_bb_tasks_w_match)

We can also use `match_labels`, which selects samples that contain at least one label matching the specified criteria. 

It does not filter the labels themselves; instead, it includes or excludes entire samples based on whether they have a matching label. For example, to keep only samples that have at least one detection with missing action types:


In [ ]:
# Create a view with missing action types

no_bb_action = gui_dataset.match_labels("detections", F("action_type") == " ")

gui_dataset.save_view("no_bb_action", no_bb_action)

This keeps only the samples with at least one matching label, but does not alter the labels within those samples.  [Views Cheat Sheet](https://docs.voxel51.com/cheat_sheets/views_cheat_sheet.html), [API Reference](https://docs.voxel51.com/api/fiftyone.core.collections.html)



**When to use which:**

- Use `filter_labels` when you want to keep all samples but only certain labels within them. That is, you want to **filter labels within samples**. 

- Use `match_labels` when you want to keep only those samples that have at least one label matching your criteria. That is, you want to **filters samples based on their labels**.

For more details and examples, see the [Filtering Cheat Sheet](https://docs.voxel51.com/cheat_sheets/filtering_cheat_sheet.html) and [Views Cheat Sheet](https://docs.voxel51.com/cheat_sheets/views_cheat_sheet.html).

In [ ]:
# Create a view with missing labels

no_bb_labels = gui_dataset.filter_labels("detections", F("label") == "")

gui_dataset.save_view("no_bb_labels", no_bb_labels)

You can also do the same with keypoints as well:

In [16]:
# Create a view with missing labels

no_kp_tasks = gui_dataset.filter_labels("keypoints", F("task_description") == "")

gui_dataset.save_view("no_kp_tasks", no_kp_tasks)

## Analyzing Bounding Box Areas

To compute the area of bounding boxes in FiftyOne, you can use a [ViewExpression]((https://docs.voxel51.com/api/fiftyone.core.expressions.html).). Bounding boxes are stored in the format [top-left-x, top-left-y, width, height], where width and height are relative to the image size (values between 0 and 1).

This computes the area as width × height in relative coordinates (as a fraction of the image).

You can use these expressions in view stages like `filter_labels` to filter detections by area.

In [17]:
from fiftyone import ViewField as F

rel_bbox_area = F("bounding_box")[2] * F("bounding_box")[3]

im_width, im_height = F("$metadata.width"), F("$metadata.height")

gui_dataset.set_field("detections.detections.relative_bbox_area", rel_bbox_area).save()

In [18]:
gui_dataset.skip(51).first()

<SampleView: {
    'id': '68a89a1de619a3e80badb873',
    'media_type': 'image',
    'filepath': '/Users/harpreetsahota/workspace/visual_agents_workshop/session_2/data/2025-07-24_17-20-32.png',
    'tags': [],
    'metadata': <ImageMetadata: {
        'size_bytes': None,
        'mime_type': None,
        'width': 1064,
        'height': 1080,
        'num_channels': None,
    }>,
    'created_at': datetime.datetime(2025, 8, 22, 16, 26, 5, 464000),
    'last_modified_at': datetime.datetime(2025, 8, 22, 16, 34, 57, 397000),
    'keypoints': <Keypoints: {
        'keypoints': [
            <Keypoint: {
                'id': '68a89a1de619a3e80badb840',
                'attributes': {},
                'tags': [],
                'label': 'click',
                'points': [[0.04077669902912621, 0.4660212320574163]],
                'confidence': None,
                'index': None,
                'visible': [2],
                'supercategory': 'interaction',
                'iscrowd': 0,

## Counting UI Elements and Interaction Points

We can also easily store counts of UI elements and interaction points for each sample in our GUI dataset. 

This makes it easier to analyze the complexity and interaction density of different interfaces.

`ViewField (F)` provides a powerful way to extract and manipulate data from our samples. 

We're using the `.length()` method to count the number of detections and keypoints in each sample, then storing these counts as new fields in our dataset.

These count fields enable us to:

- Filter samples by UI complexity (e.g., find screens with many UI elements)

- Analyze the relationship between UI elements and interaction points

- Create visualizations based on interaction density

- Sort and group samples by complexity

In [19]:
from fiftyone import ViewField as F

box_counts = gui_dataset.values(F("detections.detections").length())

gui_dataset.set_values("box_counts", box_counts)

In [20]:
point_counts = gui_dataset.values(F("keypoints.keypoints").length())

gui_dataset.set_values("point_counts", point_counts)

In [21]:
gui_dataset.skip(51).first()

<SampleView: {
    'id': '68a89a1de619a3e80badb873',
    'media_type': 'image',
    'filepath': '/Users/harpreetsahota/workspace/visual_agents_workshop/session_2/data/2025-07-24_17-20-32.png',
    'tags': [],
    'metadata': <ImageMetadata: {
        'size_bytes': None,
        'mime_type': None,
        'width': 1064,
        'height': 1080,
        'num_channels': None,
    }>,
    'created_at': datetime.datetime(2025, 8, 22, 16, 26, 5, 464000),
    'last_modified_at': datetime.datetime(2025, 8, 22, 16, 35, 36, 122000),
    'keypoints': <Keypoints: {
        'keypoints': [
            <Keypoint: {
                'id': '68a89a1de619a3e80badb840',
                'attributes': {},
                'tags': [],
                'label': 'click',
                'points': [[0.04077669902912621, 0.4660212320574163]],
                'confidence': None,
                'index': None,
                'visible': [2],
                'supercategory': 'interaction',
                'iscrowd': 0,

## Advanced Embedding Analysis with FiftyOne

FiftyOne's Brain module provides powerful capabilities for working with embeddings. 

Here are some of the most powerful things you can do:

#### Visualizing Embeddings with Dimensionality Reduction

You can project high-dimensional embeddings into 2D or 3D space using techniques like:
- **UMAP**: Non-linear dimensionality reduction that preserves local relationships
- **t-SNE**: Excellent for visualizing clusters in high-dimensional data
- **PCA**: Linear dimensionality reduction that captures maximum variance

This visualization allows you to:
- Discover patterns and clusters in your GUI dataset
- Identify outliers or unusual interactions
- Compare different interaction types visually
- Explore the feature space of your data


Here's how to compute embeddings for GUI screenshots. We'll use a pretrained model from the FiftyOne Model Zoo. For actual GUI analysis, you might want a model trained on UI elements.

In [22]:
import fiftyone.brain as fob
import fiftyone.zoo as foz

clip_model = foz.load_zoo_model("open-clip-torch")

results = fob.compute_visualization(
    gui_dataset,
    model=clip_model,
    embeddings="clip_embeddings",
    method="umap",  # "umap", "tsne", "pca", etc
    brain_key="clip_viz"
)

/opt/anaconda3/envs/fiftyone/lib/python3.11/site-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


Computing embeddings...


INFO:fiftyone.brain.internal.core.utils:Computing embeddings...


 100% |█████████████████| 739/739 [23.8s elapsed, 0s remaining, 18.9 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 739/739 [23.8s elapsed, 0s remaining, 18.9 samples/s]      


Generating visualization...


INFO:fiftyone.brain.visualization:Generating visualization...
/opt/anaconda3/envs/fiftyone/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


UMAP( verbose=True)
Fri Aug 22 11:36:56 2025 Construct fuzzy simplicial set
Fri Aug 22 11:36:57 2025 Finding Nearest Neighbors
Fri Aug 22 11:36:59 2025 Finished Nearest Neighbor Search
Fri Aug 22 11:36:59 2025 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Fri Aug 22 11:37:00 2025 Finished embedding



### 2. Similarity Search

Build powerful similarity indexes for:
- Finding visually similar GUI screens
- Identifying related interaction patterns
- Retrieving nearest neighbors for any sample

FiftyOne supports multiple backends including:
- Sklearn (default, in-memory)
- Vector databases (Qdrant, Pinecone, Redis, Milvus, etc.)
- Document stores (MongoDB, Elasticsearch)

In [23]:
results = fob.compute_similarity(
    gui_dataset,
    backend="sklearn",  # Fast sklearn backend
    brain_key="clip_sim", 
    embeddings="clip_embeddings"
)


### Computing Representativeness

Create diverse subsets of your GUI dataset by:
- Selecting samples that best represent the entire distribution
- Ensuring coverage of all interaction types and patterns
- Avoiding redundancy in your selected examples


In [24]:
results = fob.compute_representativeness(
    gui_dataset,
    similarity_index="clip_sim"
)

Retrieving embeddings from similarity index...


INFO:fiftyone.brain.internal.core.utils:Retrieving embeddings from similarity index...


Computing representativeness...


INFO:fiftyone.brain.internal.core.representativeness:Computing representativeness...


Computing clusters for 739 embeddings; this may take awhile...


INFO:fiftyone.brain.internal.core.representativeness:Computing clusters for 739 embeddings; this may take awhile...


Representativeness computation complete


INFO:fiftyone.brain.internal.core.representativeness:Representativeness computation complete


# Patch views in FiftyOne
 
Patch views are a powerful feature in FiftyOne that allow you to:
- Extract regions of interest (like bounding boxes) as standalone samples
- Work with these regions directly instead of the full images
- Maintain all metadata and annotations from the parent sample
- Perform analysis on specific UI elements rather than entire screens

When we convert detections to patches (as in the next cell), each bounding box becomes its own sample with all associated metadata like task descriptions,action types, and element information preserved.


In [ ]:
box_patches = gui_dataset.to_patches("detections")
box_patches=box_patches.clone(name="box_patches")
box_patches.persistent=True

In [ ]:
box_patches.skip(51).first()

In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Initialize the embedding model
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True, device="mps")

# Iterate through patches and compute embeddings
for patch in tqdm(box_patches, desc="Computing embeddings"):
    # Construct prompt from the three fields
    task_desc = patch.detections.task_description if patch.detections.task_description else ""
    action_type = patch.detections.action_type if patch.detections.action_type else ""
    element_info = patch.detections.element_info if patch.detections.element_info else ""
    
    # Combine into prompt
    prompt = f"clustering: Task: {task_desc}; Action: {action_type}; Element: {element_info}"
    
    # Compute embedding
    embedding = model.encode(prompt)
    
    # Add as new field
    patch['text_embedding'] = embedding.tolist()
    patch.save()

print("Done! Embeddings added to all patches.")

In [ ]:
import fiftyone.brain as fob 

results = fob.compute_visualization(
    box_patches,
    embeddings="text_embedding",
    method="umap",  # "umap", "tsne", "pca", etc
    brain_key="text_viz"
)

Let's get more hands on with the FiftyOne app!

In [ ]:
import fiftyone as fo

fo.launch_app(gui_dataset)

# or run this in the terminal: fiftyone app launch

Now, we can push our dataset to the Hugging Face Hub.

You will need a Hugging Face token, learn how you can get one [here](https://huggingface.co/docs/hub/en/security-tokens#how-to-manage-user-access-tokens).

You'll then need to login to Hugging Face, which you can by entering the following in your terminal:

`hf auth login`

And then paste your token.

In [ ]:
import fiftyone.utils.huggingface as fouh

fouh.push_to_hub(
    gui_dataset,
    "FiftyOne-GUI-Grounding-Train"
)